# 0.Initial 

In [11]:
import re
import schedule

import time
from datetime import datetime, timedelta

import json

import requests

from bs4 import BeautifulSoup

import mysql.connector

pattern = re.compile(r'jQuery\d+_\d+\((.*?)\);')

def get_title(text):
    return text.split("】", 1)[0].split("【", 1)[-1].strip()

def get_text(text):
    return text.split("】", 1)[-1].strip()

def get_source(text):
    return text.rsplit("（", 1)[-1].rsplit("）", 1)[0].strip()

def get_text_from_source(text):
    return text.rsplit("（", 1)[0].strip()

def get_section(text):
    return text.split("|", 1)[0].strip()

def get_text_from_section(text):
    return text.split("|", 1)[-1].strip()

def get_section_chinese(text):
    return text.split("｜", 1)[0].strip()

def get_text_from_section_chinese(text):
    return text.split("｜", 1)[-1].strip()

def get_section_special(text):
    return text.split("︱", 1)[0].strip()

def get_text_from_section_special(text):
    return text.split("︱", 1)[-1].strip()

def print_scrape_progress(time_last, num):
    print("Data scraped until %s. %d was inserted into MySQL." % (time_last, num), end="\r")
    
def exit_loop_condition(time_this, time_now, time_delta):
    return time_this <= time_now - timedelta(hours = time_delta)

def start_mysql():
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="password",
        database="NEWS",
        auth_plugin='mysql_native_password'
    )

    mycursor = mydb.cursor()
    return mycursor, mydb

def insert_mysql(val, mycursor, mydb, table):
    sql = "INSERT INTO " + table + " (originalID, time, title, source, section, text) VALUES (%s, %s, %s, %s, %s, %s)"
    mycursor.executemany(sql, val)
    mydb.commit()

def close_mysql(mycursor, mydb):
    mycursor.close()
    mydb.close()

headers = """
Accept: */*
Accept-Encoding: gzip, deflate, br, zstd
Accept-Language: zh-CN,zh;q=0.9,en;q=0.8
Connection: keep-alive
Cookie: v=A4YJcvo4HO9K8MiLKiJJdN9r0XcN58qhnCv-BXCvcqmEcygp2HcasWy7ThBD
Host: news.10jqka.com.cn
Referer: https://news.10jqka.com.cn/realtimenews.html
Sec-Fetch-Dest: empty
Sec-Fetch-Mode: cors
Sec-Fetch-Site: same-origin
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36
X-Requested-With: XMLHttpRequest
hexin-v: A4YJcvo4HO9K8MiLKiJJdN9r0XcN58qhnCv-BXCvcqmEcygp2HcasWy7ThBD
sec-ch-ua: "Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"
sec-ch-ua-mobile: ?0
sec-ch-ua-platform: "macOS"
"""
headers = dict([i.split(": ") for i in headers.split("\n") if i !=""])

# 1. SINA

In [2]:
def get_sina_news(time_delta=24):
    datetime_now = datetime.now()
    mycursor, mydb = start_mysql()

    url = "https://zhibo.sina.com.cn/api/zhibo/feed?callback=jQuery0_%s&page=1&page_size=1&zhibo_id=152&tag_id=0" % (datetime_now.strftime("%s"))
    response = requests.get(url)
    match = pattern.search(response.text)
    json_content = match.group(1)
    data = json.loads(json_content)
    id_now = data["result"]["data"]["feed"]["list"][0]["id"]

    page = 1
    while True:
        url = "https://zhibo.sina.com.cn/api/zhibo/feed?callback=jQuery0_%s&page=%d&page_size=100&zhibo_id=152&tag_id=0&id=%d&type=1" % (datetime_now.strftime("%s"), page, id_now)
        response = requests.get(url)
        match = pattern.search(response.text)
        json_content = match.group(1)
        data = json.loads(json_content)
        data_list = data["result"]["data"]["feed"]["list"]
        val = []
        for i in data_list:
            if i["rich_text"][0] == "【":
                val.append([i["id"], i["create_time"], get_title(i["rich_text"]), "", "", get_text(i["rich_text"])])
            else:
                val.append([i["id"], i["create_time"], "", "", "", i["rich_text"]])

        insert_mysql(val, mycursor, mydb, "sina")

        time_last = data_list[-1]["create_time"]
        print_scrape_progress(time_last, mycursor.rowcount)
        if exit_loop_condition(datetime.strptime(time_last, '%Y-%m-%d %H:%M:%S'), datetime_now, time_delta):
            break
        else:
            page += 1

    close_mysql(mycursor, mydb)

# 2. 10JQKA

In [3]:
def get_10jqka_news(time_delta=24):
    datetime_now = datetime.now()
    mycursor, mydb = start_mysql()

    page = 1
    while True:
        url = "https://news.10jqka.com.cn/tapp/news/push/stock/?page=%d&tag=&track=website&pagesize=100" % page
        response = requests.get(url, headers=headers)
        data_list = response.json()["data"]["list"]

        val = [[i["id"], datetime.fromtimestamp(int(i["ctime"])), i["title"], "", "", i["digest"]] for i in data_list]
        insert_mysql(val, mycursor, mydb, "10jqka")

        time_last = datetime.fromtimestamp(int(data_list[-1]["ctime"]))
        print_scrape_progress(time_last.strftime('%Y-%m-%d %H:%M:%S'), mycursor.rowcount)
        if exit_loop_condition(time_last, datetime_now, time_delta):
            break
        else:
            page = page + 1

    close_mysql(mycursor, mydb)

# 3. WSCN

In [7]:
def get_wscn_news(time_delta=24):
    mycursor, mydb = start_mysql()
    datetime_now = datetime.now()

    url = "https://api-one-wscn.awtmt.com/apiv1/content/lives?channel=global-channel&client=pc&limit=100"
    response = requests.get(url)
    data = response.json()["data"]

    val = []
    for i in data["items"]:
        if i["content_text"][-1] == "）":
            val.append([i["id"], datetime.fromtimestamp(int(i["display_time"])), 
                        i["title"], get_source(i["content_text"]), "",
                        get_text_from_source(i["content_text"])])
        else:
            val.append([i["id"], datetime.fromtimestamp(int(i["display_time"])), 
                        i["title"], "", "",
                        i["content_text"]])

    insert_mysql(val, mycursor, mydb, "wscn")

    next_cursor = data["next_cursor"]
    time_last = datetime.fromtimestamp(int(next_cursor))
    print_scrape_progress(time_last.strftime('%Y-%m-%d %H:%M:%S'), mycursor.rowcount)

    while True:
        url = "https://api-one-wscn.awtmt.com/apiv1/content/lives?channel=global-channel&client=pc&limit=100&cursor=%s" % next_cursor
        response = requests.get(url)
        data = response.json()["data"]

        val = []
        for i in data["items"]:
            if "（" in i["content_text"]:
                val.append([i["id"], datetime.fromtimestamp(int(i["display_time"])), 
                            i["title"], get_source(i["content_text"]), "",
                            get_text_from_source(i["content_text"])])
            else:
                val.append([i["id"], datetime.fromtimestamp(int(i["display_time"])), 
                            i["title"], "", "", 
                            i["content_text"]])

        insert_mysql(val, mycursor, mydb, "wscn")

        next_cursor = data["next_cursor"]
        time_last = datetime.fromtimestamp(int(next_cursor))
        print_scrape_progress(time_last.strftime('%Y-%m-%d %H:%M:%S'), mycursor.rowcount)
        if exit_loop_condition(time_last, datetime_now, time_delta):
            break

    close_mysql(mycursor, mydb)

# 4. NDB

In [5]:
def get_ndb_news(time_delta=24):
    mycursor, mydb = start_mysql()
    datetime_now = datetime.now()

    page = 1
    while True:
        url = "https://www.nbd.com.cn/columns/3/page/%d" % page

        response = requests.get(url)
        soup = BeautifulSoup(response.text)

        data = soup.find("div", {"class": "g-list-text"})
        dates = [i.text.strip() for i in data.find_all("p", {"class": "u-channeltime"})]
        this_page_ids = [[j.find("a")["href"] for j in i.find_all("li", {"class": "u-news-title"})] for i in data.find_all("ul", {"class": "u-news-list"})]
        this_page_news = [[[t.strip() for t in j.text.strip().split("\n\n")] for j in i.find_all("li", {"class": "u-news-title"})] for i in data.find_all("ul", {"class": "u-news-list"})]
        for index, this_part_news in enumerate(this_page_news):
            val = []
            for i, this_news in enumerate(this_part_news):
                if "|" in this_news[0]:
                    val.append([this_page_ids[index][i], dates[index] + " " + this_news[1], "", "", 
                                get_section(this_news[0]), get_text_from_section(this_news[0])])
                elif "｜" in this_news[0]:
                    val.append([this_page_ids[index][i], dates[index] + " " + this_news[1], "", "", 
                                get_section_chinese(this_news[0]), get_text_from_section_chinese(this_news[0])])
                elif "︱" in this_news[0]:
                    val.append([this_page_ids[index][i], dates[index] + " " + this_news[1], "", "", 
                                get_section_special(this_news[0]), get_text_from_section_special(this_news[0])])
                else:
                    val.append([this_page_ids[index][i], dates[index] + " " + this_news[1], "", "", 
                            "", this_news[0]])
            insert_mysql(val, mycursor, mydb, "ndb")

        time_last = val[-1][1]

        print_scrape_progress(time_last, mycursor.rowcount)
        if exit_loop_condition(datetime.strptime(time_last, '%Y-%m-%d %H:%M:%S'), datetime_now, time_delta):
            break
        else:
            page += 1


# Schedule Running

In [13]:
def scrape_news(time_delta=24):
    get_sina_news(time_delta)
    get_10jqka_news(time_delta)
    get_wscn_news(time_delta)
    get_ndb_news(time_delta)

In [9]:
schedule.every().day.at("23:59").do(scrape_news, 24)

while True:
    schedule.run_pending()
    time.sleep(60)